# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep
import sys
# sys.path.append('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..') # For Windows import
os.chdir('..' + sep_local + '..' + sep_local +'..') # For Linux import
print(sep_local)
print(os.getcwd())

In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='atari_pacman'

In [4]:
images_dir = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux
#images_dir = 'C:\\projects\\pokemon\DS06\\'
validation_percentage = 25
valid_format = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [6]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose = 0
)


### inputs_shape changed to (EPIS_LEN, ) + image_size due to class_mode='episode' when data is loaded

In [7]:
scale=1
image_size=(160//scale, 210//scale, 3)

batch_size = 10
EPIS_LEN = 10
EPIS_SHIFT = 5

inputs_shape= (EPIS_LEN, ) + image_size
latents_dim = 3
intermediate_dim = 30

In [8]:
#we created new  class_mode  episode_flat witch returns 
#da.from_array(np.reshape(batch_x, (-1,)+self.image_shape )), da.from_array(np.reshape(batch_gt, (-1,)+self.image_shape))

### Class_mode is episode because time dimention "TimeDistributed" of the model

In [9]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode='episode',
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

  INFO     | Found 1177538 training files


  INFO     | Found 383441 validation files


In [10]:
import tensorflow as tf
import numpy as np

### #Transformative changed output shapes of Dataset Generators 
from (batch_size, EPIS_LEN, ) + image_size)
to (batch_size* EPIS_LEN, ) + image_size) 

In [11]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
                  )
)


In [12]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [13]:
_instance_scale = 1.0

In [14]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
inputs_shape

(10, 160, 210, 3)

In [16]:
next(training_generator)[0].shape

(10, 10, 160, 210, 3)

# Model's Layers definition

In [17]:
# TdDense is for timedistributed dense model

In [18]:
# tdDense = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(**kwds))

In [19]:
# c = 3
# enc_lays = [
#     tdConv(filters=c, kernel_size=3, strides=(2, 2), activation='relu'),
#     tdConv(filters=2*c, kernel_size=3, strides=(2, 2), activation='relu'),
#     tf.keras.layers.Flatten(),
#     # No activation
#     tf.keras.layers.Dense(latents_dim)
# ]


# dec_lays = [
#     tf.keras.layers.Dense(units=c**3, activation=tf.nn.relu),
#     tf.keras.layers.Reshape(target_shape=(1, c, c, c)),
#     tdDeConv(filters=2*c, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
#     tdDeConv(filters=c, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
#     # No activation
#     tdDeConv(filters=1, kernel_size=3, strides=(1, 1), padding="SAME"),    
# ]

In [20]:
tdConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(**kwds))

In [21]:
tdDeConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2DTranspose(**kwds))

### #Transformative Dense layer units=latents_dim*EPIS_LEN meaning 30 due to uniformity between all networks  target_shape=(EPIS_LEN,) + c

In [22]:
units=30
c=(image_size[0]//4, image_size[1]//6, intermediate_dim//2) # now 4x and 6x smaller since kernels are 2 and 3 
enc_lays = [
    tdConv(filters=units, kernel_size=3, strides=(2, 2), activation='relu'),
    tdConv(filters=units//5, kernel_size=3, strides=(2, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(units=latents_dim*EPIS_LEN)
]

dec_lays = [
    tf.keras.layers.Dense(units=np.product((EPIS_LEN,) + c), activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(EPIS_LEN,) + c),
    tdDeConv(filters=units//5, kernel_size=3, strides=(2, 3), padding="SAME", activation='relu'),
    tdDeConv(filters=units, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tdDeConv(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition


In [23]:
import os
model_name = 'Atari_AE_TimeDist_Conv'
#experiments_dir='..'+sep_local+'..'+sep_local+'..'+sep_local+'experiments'+sep_local + model_name

experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [24]:
from training.autoencoding_basic.transformative.AE import autoencoder as AE

### #Transformative 'outputs_shape':latents_dim*EPIS_LEN,

In [25]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim*EPIS_LEN,
        'layers': enc_lays
    }
    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim*EPIS_LEN,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [26]:
from os.path import abspath
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)
absolute = abspath(experiments_dir)
print("experiments_dir",absolute)
print("Current working dir",os.getcwd())

Restore_dir /home/azeghost/git/new_GAN_models/Generative_Models/experiments/Atari_AE_TimeDist_Conv/var_save_dir
experiments_dir /home/azeghost/git/new_GAN_models/Generative_Models/experiments/Atari_AE_TimeDist_Conv
Current working dir /home/azeghost/git/new_GAN_models/Generative_Models


In [27]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    episode_len= EPIS_LEN, 
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 10, 160, 210, 3)] 0         
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 79, 104, 30)   840       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 39, 34, 6)     1626      
_________________________________________________________________
flatten (Flatten)            (None, 79560)             0         
_________________________________________________________________
dense (Dense)                (None, 30)                2386830   
_________________________________________________________________
batch_normalization (BatchNo (None, 30)                120       
_________________________________________________________________
dropout (Dropout)            (None, 30)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 30)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 210000)            6510000   
_________________________________________________________________
reshape (Reshape)            (None, 10, 40, 35, 15)    0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 80, 105, 6)    816       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 10, 160, 210, 30)  1650      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 160, 210, 3)   813       
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 160, 210, 3)   1

  WARNING  | None


In [28]:
# ae.compile(metrics=None)
ae.compile()

Model: "Atari_AE_TimeDist_Conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 10, 160, 210, 3)] 0         
_________________________________________________________________
inference (Model)            (None, 30)                2389416   
_________________________________________________________________
generative (Model)           (None, 10, 160, 210, 3)   6513291   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 10, 160, 210, 3)] 0         
Total params: 8,902,707
Trainable params: 8,902,641
Non-trainable params: 66
_________________________________________________________________
None


# Callbacks

In [29]:
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [31]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [32]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

Csv_dir /home/azeghost/git/new_GAN_models/Generative_Models/experiments/Atari_AE_TimeDist_Conv/csv_dir/Atari_AE_TimeDist_Conv.csv


In [33]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

Image_gen_dir /home/azeghost/git/new_GAN_models/Generative_Models/experiments/Atari_AE_TimeDist_Conv/image_gen_dir


### #Transformative latents_shape should be  latents_dim* EPIS_LEN

In [34]:
sg = SampleGeneration(latents_shape=latents_dim* EPIS_LEN, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

# Model Training

In [35]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=2,
    epochs=2, 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=2
)

Train for 2 steps, validate for 2 steps
Epoch 1/2


ResourceExhaustedError: OOM when allocating tensor with shape[500,30,161,211] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2DBackpropInput] name: generative/time_distributed_3/conv2d_transpose_1/conv2d_transpose/

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)

### Stacked inputs outputs and predictions 

In [ ]:
from training.generators.image_generation_testing import predict_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'predictions')
create_if_not_exist(save_dir)

predict_from_a_batch(ae, testing_generator, save_dir)